# Imports

In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

import matplotlib.pyplot as plt
import time
import os
import copy


# Set device

In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loading the training dataset

In [80]:
data_transform = transforms.Compose([
        #TODO:think a better transformation pipeline
        #naive transformation
        transforms.Resize((64,64)),
        transforms.ToTensor()
    ])

In [81]:
train_dir='./dataset/GTSRB/train/original'

train_dataset = datasets.ImageFolder(train_dir,data_transform)

train_loader = DataLoader(train_dataset, batch_size=200,shuffle=True, num_workers=0)

n_total_steps = len(train_loader)
train_size = len(train_dataset)
class_names = train_dataset.classes

print(class_names)

['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042']


In [82]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
    
            # forward
            with torch.set_grad_enabled(True):
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
    
                # backward + optimize only if in training phase
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                if (i+1) % 200 == 0:
                    print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
    
            # statistics
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
            #to update weights
            scheduler.step()
    
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
    
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            'train', epoch_loss, epoch_acc))
    
    print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

# ResNet18 backbone

In [ ]:
#### ConvNet as fixed feature extractor ####
# freeze all the network parameters except the final layer settings, to not compute the gradients backward
model_conv = torchvision.models.resnet18(weights=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 43)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------


# Save the model

In [ ]:
print('Finished Training')
PATH = './first_trained_model.pth'
torch.save(model_conv.state_dict(), PATH)